In [1]:
import db_conn
import pandas as pd
import re
pd.options.display.max_columns = None

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 부작용
추출된 전체 부작용 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_ads():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(d.tt_ad))) AS t_ad
    FROM
    (SELECT id, tt_ad
    FROM 
        (SELECT * 
        FROM article_table_sentences_m
        WHERE table_id IN 
        (SELECT id FROM article_tables WHERE final_topic IS NOT NULL)) M,
        UNNEST(STRING_TO_ARRAY(t_ad, ' , ')) AS tt_ad
    WHERE (tt_ad='') is false) as d;
    """)
    return cur.fetchall()

def get_ad_dictionary():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(LOWER(llt_name))) AS t_ad
    FROM meddra_llt_181022 M;
    """)
    return cur.fetchall()

In [4]:
ads = pd.DataFrame(get_total_ads())
# ads = pd.read_csv('./distinct_ads.tsv', sep='\t')

In [5]:
# ads.to_csv('./ads.csv', sep=',')

In [6]:
ads.columns=['name']

In [7]:
ads['refined'] = ''
ads.head()

,name,refined
0,odynophagia,
1,presyncope,
2,seizure,
3,myalgia,
4,aphthous ulcer,


#### 전처리 전 총 부작용의 수는 4900개 입니다

In [8]:
ads.shape

(4900, 2)

### 2. 전처리

#### A. dictionary를 참고하여, 정확히 일치하는 부작용 명은 전처리 없이 포함합니다.

In [9]:
dictionary = pd.DataFrame(get_ad_dictionary())
dictionary.columns = ['t_ad']
# dictionary = pd.read_csv('./dictionary_ads.tsv', sep='\t')
# dictionary = list(map(lambda x: str(x[0]).lower().replace('"', ''), dictionary))

In [10]:
dictionary.shape

(60939, 1)

In [11]:
dictionary[:10]

,t_ad
0,"""ventilation"" pneumonitis"
1,11-beta-hydroxylase deficiency
2,11-oxysteroid activity incr
3,11-oxysteroid activity increased
4,17 ketosteroids urine abnormal
5,17 ketosteroids urine abnormal nos
6,17 ketosteroids urine decreased
7,17 ketosteroids urine high
8,17 ketosteroids urine increased
9,17 ketosteroids urine low


### 부작용용어 dictionary와 정확이 매칭하면 추가 전처리 없이 확정

In [12]:
def apply_dictionary():
    for ind, ad in ads.iterrows():
    #     print(ad['name'].strip())
        name = ad['name'].strip()
        exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)==0:
            if name[0] in ['"', '.', ':', '-', '(', ')']:
                name = name[1:]
            if name[len(name)-1] in ['"', '.', ':', '-', '(', ')']:
                name = name[:len(name)-1]
    #         print(name)

            exact_mathing = dictionary.loc[dictionary.t_ad==name]

        if len(exact_mathing)>0:
            ads.loc[ind]['refined'] = name

apply_dictionary()

In [13]:
ads[ads['refined'] != ''].shape

(3993, 2)

#### A. 부작용을 포함하고 있는 구절이 추출 된 경우, 부작용 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_ads_hyj.xlsx파일을 통해 부작용명이 아니라고 지정해주신 목록을 추가하였습니다.

In [14]:
replace_with_space = ['<70%', '%', 'values)', '(', ')', '-1987', '-1993', ':', '±', '↑', 'australia-new',
                     'scale-7', 'c-statistics', '-defined', '-derived', '-like', 'like', 'non-beverage', 'beverage',
                     'scale-0', '-days', 'fluorine-18-labelled', '-specific', 'co-variate', '-guided', '≥2°',
                     '≥1', 'composting', 'daids', 'decentralised', 'dmards', 'forage', 'full-scale', 'glues',
                     'granger', 'greyscale', 'grey-scale', 'harvesting', 'immersed', 'invariably',
                     '-guidance', 'lifelon', 'mwaura', 'naito', 'nesting', 'nomad', 'nonsystemi', '≥60', 'primers',
                     'prohibited', 'sanger', '-1993', '<50', '<02']

replace_ends_with = ['-related']

In [15]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        ads.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        ads.loc[ind]['refined'] = refined
        print(row['name'])
        print(ads.loc[ind]['refined'])
    
    for end in replace_ends_with:
        if row['name'].endswith(end):
            ads.loc[ind]['refined'] = row['name'].replace(end, '')

ads = ads.reset_index(drop=True)

beverages
 s
plasmacytoma-like
plasmacytoma 
hypertension:
hypertension 
cancer-specific
cancer 
serum-sickness-like
serum-sickness 
aids-1993
aids 
migraine-days
migraine 
flu-like illness
flu  illness
melanoma-specific
melanoma 
ischemia-guided
ischemia 
flu-like symptoms
flu  symptoms
pressure-guided
pressure 
influenza-like
influenza 
acne-like
acne 
asthma-like
asthma 
headache-days
headache 
infarction-defined
infarction 
influenza-like symptoms
influenza  symptoms
binge drinking≥1
binge drinking 
(copd)
 copd 
angina-guided
angina 
breast cancer-specific
breast cancer 
flu like symptoms
flu   symptoms
myocardial infarction-defined
myocardial infarction 
aids-1987
aids 
(lbbb)
 lbbb 
lifelong
 g
variable:
variable 
stenosis-specific
stenosis 
influenza like illness
influenza   illness
pressure-derived
pressure 
hypertension-specific
hypertension 


#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

In [16]:
non_drug_ad_rel = ['-free', 'free-', ' free ', 'variable', 'wrong drug administered', 'exercise-induced',
                  'underdose', 'switch', 'massage', 'according', '-defining', 'arizona', 'controller', 'athletics',
                  'atopics', 'average', 'bachelor', 'award', '—gasoline', 'cards', 'comer', 'consisting', 'consumer',
                  'courage', 'covaria', 'coverage', 'dimers', 'distinguish', 'eavalues', 'emilia', 'multivaria',
                  'nsaids', 'preexisting', 'reaches', 'recording', 'restaura', 'spain', 'standard', 'tactics',
                  'tattoos', 'tasmania', 'telcagepant', 'testing', 'tragus', 'post-aids', 'scale–parent', 'body-mass-index',
                  'mass-cavity', 'non-melanoma skin', 'single-blind', 'non-comatose', 'non-wheezers', 'high-pitched',
                  'gadofluorine-m', 'anger-control', 'hiv-1-negative', 'pulmonary-systemic', '1apical', 'pre-infarction',
                  'non-trauma', 'stress-mass-heart', 'self-discharge', 'anti-japanese encephalitis', 'non-hypertension',
                   'non-infarct', 'post-stress', 'injury-donor', 'rate–pressure', 'non-neutropenic infections', 'non-hypertrophic',
                   'anti-tachycardia', 'scale-up', '5-fluorouracil', 'isar-smart', '18f-fluoride', 'post-acute myocardial infarction',
                   'hyper-responsiveness', 'pre-aids', 'mid-cavity', 'post-bleeding', 'scale–cognitive', 'double-blind',
                   'high-pressure', 'post-endocarditis', 'peri-procedural complications', 'modulation-blind', 'pre-shock',
                   'pre-angina', 'diphtheria–pertussis–tetanus', 'australia–new', 'q10values', 'pressure-time', 'non-congenital',
                   'anti-hypertensive', 'sepsis-induced', 'snaith-hamilton', 'mass–body', 'pain-to-balloon', 'pain-to-coronary',
                   'non-prostate cancer', 'cold-induced', 'non-melanoma', 'post-hypoxic', 'pre-injury', 'scale-16', 'non-atopic',
                   'non-hypoxic', 'cancer-norfolk', 'langerhans-cell', 'body-mass', 'double-blinding', 'fluorouracil-based',
                   'scale–child', 'fat-mass', 'nifurtimox-eflornithine', 'no adverse events'] #'non'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [17]:
for ind, row in ads.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        ads.drop(ind, inplace=True)
        print('===============================')

ads = ads.reset_index(drop=True)

infarction-free
coverage
no adverse events
covariable
single-blind
aids-defining
non-traumatic
multivariate-adjusted
pre-infarction
variable;
post-stress
multivariable–adjusted
switching
non-neutropenic infections
scale-up
isar-smart
post-acute myocardial infarction
hyper-responsiveness
preexisting
atopics
average
post-bleeding
scale–cognitive
bivariable
double-blind
peri-procedural complications
testing
non-melanoma skin
d-dimers
non-congenital
sepsis-induced
mass–body
pain-to-balloon
switched
non-melanoma
switch
body-mass
switchmrk
fluorouracil-based
fat-mass
consisting
telcagepant
distinguish
nifurtimox-eflornithine
metastasis-free
burn—gasoline
multivariate
multivariable
pain-progression-free
courage
univariable
standards
thyroxine free decreased
multivariable+
recording
covaria
massage
non-melanomatous
leukaemia-free
switchback
variable
according
seizure-free
dependent-variable
wrong drug administered
nsaids
underdosed
arrhythmia-free
myocardial infarction-free
spain
variables
var

### 파일 중에 부작용이 아니라고 표기해주신 것을 추가로 제거

In [18]:
replace_excel = pd.read_excel('./need_to_refined_ads_hyj.xlsx')
replace_excel.columns = ['key', 'replace_with', '-']
replace_excel = replace_excel[['key', 'replace_with']]
replace_excel.drop_duplicates(inplace=True)
replace_excel.head()

,key,replace_with
0.0,anuloaortic ectasia,annuloaortic ectasia
14.0,epispadias-classical,epispadias
24.0,broncho-pneumonia,broncho pneumonia
40.0,cardio-myopathy,cardiomyopathy
50.0,hypertension-stroke,Hypertension + stroke


In [19]:
ads.shape

(4820, 2)

In [20]:
for ind, cura in replace_excel[replace_excel.replace_with.str.find('부작용 아님')>=0].iterrows():
#     print(cura['name'])
    need_to_drop = ads[ads.name.str.lower().str.find(cura['key'].lower())>=0]
    for i_n, d_n in need_to_drop.iterrows():
        ads.drop(i_n, inplace=True)

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 4733 남습니다

In [21]:
ads.shape

(4733, 2)

### 복잡 부작용, 스펠링 미스 등을 선생님께서 주신 파일로 정제

In [22]:
def replace_with_manual_curation(ads):
    for ind, row in replace_excel[replace_excel.replace_with.str.find('부작용 아님')<0].iterrows():
        print(row)
        for d_ind, d_row in ads.loc[ads.name.str.strip()==row['key']].iterrows():
            origin_str = d_row['name'].replace(row['key'], row['replace_with'])

            ads.loc[d_ind]['refined'] = origin_str 
    return ads

ads = replace_with_manual_curation(ads)

key              anuloaortic ectasia
replace_with    annuloaortic ectasia
Name: 0.0, dtype: object
key             epispadias-classical
replace_with              epispadias
Name: 14.0, dtype: object
key             broncho-pneumonia
replace_with    broncho pneumonia
Name: 24.0, dtype: object
key             cardio-myopathy
replace_with     cardiomyopathy
Name: 40.0, dtype: object
key               hypertension-stroke
replace_with    Hypertension + stroke
Name: 50.0, dtype: object
key             mgh-adhd
replace_with        ADHD
Name: 64.0, dtype: object
key             heat-shock
replace_with         shock
Name: 77.0, dtype: object
key             neoplasm-associated
replace_with               neoplasm
Name: 86.0, dtype: object
key               neoangiogenesis-inflammation
replace_with    Neoangiogenesis + inflammation
Name: 114.0, dtype: object
key               dementia-motor neuron disease
replace_with    Dementia + motor neuron disease
Name: 132.0, dtype: object
key             v

Name: 1861.0, dtype: object
key             generalised tonic-clonic seizures
replace_with     generalised tonic-clonic seizure
Name: 1879.0, dtype: object
key             factor-stemi
replace_with           STEMI
Name: 1883.0, dtype: object
key             dementia–symptom
replace_with            dementia
Name: 1897.0, dtype: object
key             stenosis≥50
replace_with       stenosis
Name: 1951.0, dtype: object
key             neuropathy-rasch-built
replace_with                neuropathy
Name: 1955.0, dtype: object
key             non-vertebral fractures
replace_with                  fractures
Name: 1958.0, dtype: object
key             fibro-lipomatosis
replace_with          lipomatosis
Name: 1981.0, dtype: object
key             non-epileptic seizures
replace_with     Non-epileptic seizure
Name: 1986.0, dtype: object
key             tetanus-toxoid
replace_with           tetanus
Name: 2010.0, dtype: object
key             hads-anxiety
replace_with         anxiety
Name: 2029.0, dt

key             irritable-bowel
replace_with    Irritable bowel
Name: 3869.0, dtype: object
key             ischaemia-driven
replace_with           ischaemia
Name: 3877.0, dtype: object
key             mismatch-hyperkinesia
replace_with             hyperkinesia
Name: 3883.0, dtype: object
key             all-causedeathorhospitalizationforhf
replace_with                           heart failure
Name: 3891.0, dtype: object
key             dengue-2
replace_with      dengue
Name: 3895.0, dtype: object
key             scale-depression
replace_with          depression
Name: 3910.0, dtype: object
key             questionnaire-hepatitis c
replace_with                  hepatitis c
Name: 3919.0, dtype: object
key             atrial fibrillation-timi
replace_with         atrial fibrillation
Name: 3929.0, dtype: object
key               diabetes mellitustype2
replace_with    Type 2 diabetes mellitus
Name: 3938.0, dtype: object
key                     hyper-chol
replace_with    Hypercholesteremia
Na

key             stemi–rt
replace_with       stemi
Name: 6279.0, dtype: object
key             pain–rash
replace_with         pain
Name: 6328.0, dtype: object
key             ataxia-telangiectasia
replace_with    Ataxia telangiectasia
Name: 6362.0, dtype: object
key             hepatitis-c
replace_with    hepatitis c
Name: 6477.0, dtype: object
key             dilated cardiomyopathy-4
replace_with              cardiomyopathy
Name: 6488.0, dtype: object
key             adhd-dub2
replace_with         adhd
Name: 6505.0, dtype: object
key             acute myocardial infarction-4
replace_with      acute myocardial infarction
Name: 6531.0, dtype: object
key             state-trait-anger-expression
replace_with                           anger
Name: 6564.0, dtype: object
key             1injury
replace_with     injury
Name: 6589.0, dtype: object
key                      post-traumatic-stress
replace_with    Post-traumatic stress disorder
Name: 6600.0, dtype: object
key             ace-obesity


key               hypertension-heart failure
replace_with    Hypertension + heart failure
Name: 8479.0, dtype: object
key             fall-from
replace_with         fall
Name: 8495.0, dtype: object
key             brain-death
replace_with    Brain death
Name: 8516.0, dtype: object
key                      respiratory-distress
replace_with    Respiratory distress syndrome
Name: 8518.0, dtype: object
key               sinusitis-otitis
replace_with    Sinusitis + otitis
Name: 8565.0, dtype: object
key             infarction-placebo
replace_with            infarction
Name: 8569.0, dtype: object
key               confusion-ataxia
replace_with    Confusion + Ataxia
Name: 8632.0, dtype: object
key             major-stroke
replace_with          stroke
Name: 8652.0, dtype: object
key             frontotemporal dementia-motor
replace_with          Frontotemporal dementia
Name: 8693.0, dtype: object
key             blister-fluid
replace_with          blister
Name: 8737.0, dtype: object
key       

### 여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [23]:
def separate_unique_ads(ads):
    additional_ads = []

    for ind, row in ads[ads.refined.str.find('+')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split('+')))

        if len(processed)>0:
            ads.loc[ind].refined = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'refined':pro})

    for ind, row in ads[ads.refined.str.find(',')>=0].iterrows():
        processed = list(map(lambda x: x.strip(), row['refined'].split(',')))

        if len(processed)>0:
            ads.loc[ind].refined = processed[0]
            for pro in processed[1:]:

                additional_ads.append({'name':row['name'], 'refined':pro})


    ads = ads.append(pd.DataFrame(additional_ads))
    ads.drop_duplicates(inplace=True)
    return ads

ads = separate_unique_ads(ads)

In [24]:
ads.shape

(4738, 2)

### 추가 전처리가 필요한 약물 갯수

In [25]:
len(ads[(ads.refined=='')])

599

In [26]:
def get_only_string(text):
    try:
        float(text)
        return False
    except ValueError:
        return True
    
def remove_comparison(name, spliter):
    arrs = list(filter(lambda x: get_only_string(x.strip().replace('%', '')) and x.strip(), name.split(spliter)))
    if len(arrs)>0:
        return arrs
    else:
        return None

for ind, row in ads.loc[ads.name.str.find('<')>=0].iterrows():
    processed = remove_comparison(row['name'], '<')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = ''.join(processed)
# comp_ads.refined = comp_ads.name.map(lambda x: remove_comparison(x))

In [27]:
ads.loc[ads.name.str.find('<')>=0]

,name,refined
1590,values<25,values
1875,dyslipidemia<5,dyslipidemia
2234,hypertension<5,hypertension
3483,diabetes<5,diabetes
3666,values<15,values
4046,pneumonia<1,pneumonia
4722,binge drinking<1,binge drinking


여러 부작용이 하나의 레코드에 담겨있는 경우, 분리하여 입력

In [28]:
additional_ads = []

for ind, row in ads.loc[ads.name.str.find('>')>=0].iterrows():
    processed = remove_comparison(row['name'], '>')
    if processed is None:
        ads.drop(ind, inplace=True)
    else:
        ads.loc[ind].refined = processed[0]
        for pro in processed[1:]:
            print(pro)
            additional_ads.append({'name':row['name'], 'refined':pro})

ads = ads.append(pd.DataFrame(additional_ads))
ads.drop_duplicates(inplace=True)

healthy
healthy


In [29]:
ads.shape

(4740, 2)

### 앞 뒤의 -, . 제거

In [30]:
remove_list = ['.', '-related', '-', ':', '+', '–', ';', '%', 'time', 'invariably', '||']
def preprocess(name):
    if name[0] in remove_list:
        name = name[1:len(name)]
    
    if name[len(name)-1] in remove_list:
        name = name[0:len(name)-1]
    return name
    
ads.name = ads.name.map(lambda x: preprocess(x))

In [31]:
remove_exact = ["electrocardiogram changes", "discharged", "historyshock", "alternans", "nonmelanoma skin", "nonmelanoma", "values",
               "statistics", 'actuarial', 'dangerous', 'shockable', 'presence of q waves', 'australia', "topical", "systemic",
               "familial", "fluor", "variations", "towards", "slaughter", "prestroke", "variation", "adjusting", "storage", 
                "subscale", "approached", "reached", "discharges", "accidental", "morale", "australian", "teache", "accidents", 
                "overfitting", "persisting", "confusional", "typically", "existing", "scales", "yards", "typical", "australi", 
                "biomass", "onwards", "subscales", "univariatep", "unrestricted", "rewards", "variant", "mutivariate", "vegetarian", 
                "intolerance", "bivariate", "antibiotics", "adjusted", "cystatin", "hazards", "intense", "anginal", "elispot", 
                "blinded", "pregnant", "pressure", "antipsychotic", "deaths", "fluoropyrimidine", "tolerance", "variance", "cavity", 
                "fluorouracil", "transapical", "resting", "varia", "edwards", "fluorodeoxyglucose", "homocysteine", "teriflunomide", 
                "defibrillation"
               ]
for i, row in ads[ads.refined==''].iterrows():
    if row['name'].strip().lower() in remove_exact:
        ads.drop(i, inplace=True)

In [32]:
apply_dictionary()

ads.drop_duplicates(inplace=True)

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
ads.shape

(4617, 2)

In [34]:
ads[ads.refined==''].shape

(510, 2)

### 추가 전처리 필요없는 부작용들은  refined로 입력

In [35]:
# remove_list.extend([')', '('])
# spe_chars = dict((re.escape(str(k)), ' ') for k in remove_list)
# sep_pattern = re.compile("|".join(spe_chars.keys()))
# int_pattern = re.compile('[0-9]')

# for ind, row in ads.iterrows():
#     refined = sep_pattern.sub(lambda m: spe_chars[re.escape(m.group(0))], row['name'])
#     int_refined = int_pattern.search(row['name'])
    
#     if refined.strip() == '':
#         ads.drop(ind, inplace=True)
#     elif row['name'].strip() == refined.strip() and int_refined is None:
#         ads.loc[ind]['refined'] = refined
#         print(row['name'])
#         print(ads.loc[ind]['refined'])
#     print('===============================')

In [36]:
# ads.loc[ads.refined.str.len()>0].shape

In [37]:
# ads.reset_index(drop=True)

## 정체된 목록입니다.

In [38]:
ads.loc[ads.refined.str.strip()!=''].to_csv('./refined_ads_0226.csv', sep=',')

## 선생님께 확인 요청을 드리고 싶은 부분

In [39]:
ads[ads['refined'] == ''].to_csv('./need_to_refined_ads_0226.csv', sep=',')